# Object Producer

In [ ]:
import json
import yaml
import logging
import websockets
import asyncio
from kafka import KafkaProducer
from datetime import datetime
import uuid


class BinanceWebSocketProducer:
    def __init__(self, config_path: str):
        """Initialize the WebSocket producer with configuration"""
        with open(config_path, 'r') as file:
            self.config = yaml.safe_load(file)

        self._setup_logging()
        self._setup_kafka_producer()

    def _setup_logging(self):
        """Configure logging"""
        logging.basicConfig(
            level=self.config['logging']['level'],
            format=self.config['logging']['format'],
            filename=self.config['logging']['file']
        )
        self.logger = logging.getLogger('BinanceWebSocketProducer')

        # Add console output
        console_handler = logging.StreamHandler()
        console_handler.setLevel(self.config['logging']['level'])
        console_handler.setFormatter(logging.Formatter(self.config['logging']['format']))
        self.logger.addHandler(console_handler)

    def _setup_kafka_producer(self):
        """Initialize Kafka producer with configuration"""
        from configs import kafka_config
        self.producer = KafkaProducer(
            bootstrap_servers=kafka_config['bootstrap_servers'],
            security_protocol=kafka_config['security_protocol'],
            sasl_mechanism=kafka_config['sasl_mechanism'],
            sasl_plain_username=kafka_config['username'],
            sasl_plain_password=kafka_config['password'],
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            key_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        self.logger.info("Kafka producer initialized successfully")

    async def start_streaming(self):
        """Start WebSocket streaming"""
        async with websockets.connect(self.config['binance']['websocket_url']) as websocket:
            self.logger.info("Connected to Binance WebSocket")

            # Subscribe to streams
            subscribe_message = {
                "method": "SUBSCRIBE",
                "params": [f"{symbol}@trade" for symbol in self.config['binance'].get('symbols', ['btcusdt'])],
                "id": 1
            }
            await websocket.send(json.dumps(subscribe_message))
            self.logger.info(f"Sent subscription message: {subscribe_message}")

            while True:
                message = await websocket.recv()
                self.logger.info(f"Received message: {message}")

                # Parse and send to Kafka
                data = json.loads(message)
                self.producer.send(
                    self.config['topics']['raw_data'],
                    value=data,
                    key=str(uuid.uuid4())
                )
                self.producer.flush()
                self.logger.info(f"Message sent to Kafka: {data}")

    def shutdown(self):
        """Clean shutdown of the producer"""
        try:
            self.producer.flush()
            self.producer.close()
            self.logger.info("Producer shut down successfully")
        except Exception as e:
            self.logger.error(f"Error during shutdown: {str(e)}")


if __name__ == "__main__":
    producer = BinanceWebSocketProducer('config.yaml')

    async def run_producer():
        try:
            await producer.start_streaming()
        except KeyboardInterrupt:
            producer.logger.info("Received shutdown signal")
        finally:
            producer.shutdown()

    loop = asyncio.get_event_loop()
    if loop.is_running():
        # Для середовищ, де цикл вже працює
        loop.create_task(run_producer())
    else:
        loop.run_until_complete(run_producer())



# Dev consumer

In [2]:
producer = BinanceWebSocketProducer('./config.yaml')

In [3]:
producer.config['binance'].get('symbols', 'btcusdt@trade')

In [ ]:
consumer._setup_logging()

In [ ]:
# self._setup_logging()
consumer._setup_kafka_consumer()
# self._setup_kafka_producer()
# self._setup_symbol_mapping()

In [11]:
consumer._setup_kafka_producer()

In [4]:
try:
    await producer.start_streaming()
except KeyboardInterrupt:
    producer.logger.info("Received shutdown signal")

In [3]:
import json
import yaml
import logging
from kafka import KafkaConsumer, KafkaProducer
from typing import Dict, Any


def setup_logging(config: Dict[str, Any]) -> logging.Logger:
    """Configure and return logger"""
    logging.basicConfig(
        level=config['logging']['level'],
        format=config['logging']['format'],
        filename=config['logging']['file']
    )
    logger = logging.getLogger('ForexKafkaConsumer')
    return logger


def setup_kafka_consumer(config: Dict[str, Any], logger: logging.Logger) -> KafkaConsumer:
    """Initialize Kafka consumer"""
    try:
        consumer = KafkaConsumer(
            config['topics']['raw_data'],
            bootstrap_servers=config['kafka']['bootstrap_servers'],
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            **config['kafka']['consumer_config']
        )
        logger.info("Kafka consumer initialized successfully")
        return consumer
    except Exception as e:
        logger.error(f"Failed to initialize Kafka consumer: {str(e)}")
        raise


def setup_kafka_producer(config: Dict[str, Any], logger: logging.Logger) -> KafkaProducer:
    """Initialize Kafka producer for filtered streams"""
    try:
        producer = KafkaProducer(
            bootstrap_servers=config['kafka']['bootstrap_servers'],
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            **config['kafka']['producer_config']
        )
        logger.info("Kafka producer initialized successfully")
        return producer
    except Exception as e:
        logger.error(f"Failed to initialize Kafka producer: {str(e)}")
        raise


def setup_symbol_mapping(config: Dict[str, Any]) -> Dict[str, str]:
    """Create mapping of symbols to their respective topics"""
    return {
        pair['symbol'].lower(): pair['topic']
        for pair in config['topics']['currency_pairs']
    }


def process_message(
    message: Dict[str, Any],
    symbol_topics: Dict[str, str],
    producer: KafkaProducer,
    logger: logging.Logger
):
    """Process individual message and produce to appropriate topic"""
    try:
        symbol = message.get('s', '').lower()

        if symbol in symbol_topics:
            # Transform message for downstream processing
            processed_data = {
                'symbol': symbol,
                'price': float(message.get('p', 0)),
                'quantity': float(message.get('q', 0)),
                'timestamp': message.get('T', 0),
                'processing_timestamp': message.get('processing_timestamp')
            }

            # Produce to symbol-specific topic
            producer.send(
                symbol_topics[symbol],
                value=processed_data
            )
            logger.debug(f"Processed message for symbol {symbol}")
    except Exception as e:
        logger.error(f"Error processing message: {str(e)}")


def start_consuming(config_path: str):
    """Main logic for consuming and processing messages"""
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)

    logger = setup_logging(config)
    consumer = setup_kafka_consumer(config, logger)
    producer = setup_kafka_producer(config, logger)
    symbol_topics = setup_symbol_mapping(config)

    logger.info("Starting to consume messages")
    try:
        for msg in consumer:
            process_message(msg.value, symbol_topics, producer, logger)
    except KeyboardInterrupt:
        logger.info("Received shutdown signal")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
    finally:
        shutdown(consumer, producer, logger)


def shutdown(consumer: KafkaConsumer, producer: KafkaProducer, logger: logging.Logger):
    """Clean shutdown of consumer and producer"""
    try:
        consumer.close()
        producer.flush()
        producer.close()
        logger.info("Consumer and producer shut down successfully")
    except Exception as e:
        logger.error(f"Error during shutdown: {str(e)}")


if __name__ == "__main__":
    start_consuming('config.yaml')


In [4]:
from kafka import KafkaConsumer
from configs import kafka_config
import json

def safe_deserialize(data):
    return json.loads(data.decode('utf-8')) if data else None

consumer = KafkaConsumer(
    bootstrap_servers=kafka_config['bootstrap_servers'],
    security_protocol=kafka_config['security_protocol'],
    sasl_mechanism=kafka_config['sasl_mechanism'],
    sasl_plain_username=kafka_config['username'],
    sasl_plain_password=kafka_config['password'],
    value_deserializer=safe_deserialize,
    key_deserializer=safe_deserialize,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my_consumer_group_1'
)

topic_name = "currency_raw_data"

consumer.subscribe([topic_name])

print(f"Subscribed to topic '{topic_name}'")

try:
    for message in consumer:
        print(f"Raw Kafka message: {message}")
        print(f"Message key: {message.key}, Message value: {message.value}")

        data = message.value
        if data:
            print(f"Deserialized value: {data}")
        else:
            print("Message value is None or could not be deserialized.")

        # Ваші додаткові обробки тут
        # time.sleep(1)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    consumer.close()


In [2]:
import datetime
import uuid

from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType
from pyspark.sql import SparkSession
from configs import kafka_config
import os

os.environ[
    'PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'

spark = (SparkSession.builder
         .appName("KafkaStreaming")
         .master("local[*]")
         .config("spark.sql.debug.maxToStringFields", "200")
         .config("spark.sql.columnNameLengthThreshold", "200")
         .getOrCreate())


your 131072x1 screen size is bogus. expect trouble
25/01/11 15:50:09 WARN Utils: Your hostname, DESKTOP-1G0D7LM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/01/11 15:50:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/gamelt/spark/spark-3.5.1/assembly/target/scala-2.12/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gamelt/.ivy2/cache
The jars for the packages stored in: /home/gamelt/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-802b7912-44fb-4546-93d1-131cb758a035;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.10.3 in local-m2-cache
	found org.slf4j#slf4j-api;2.0.7 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in local-m2-cache
	found org.apache.hadoop#hadoop-client-api;3.3.4 in local-m2-cache
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.0 in local-m2-

In [7]:
import yaml
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    window, avg, min, max, stddev, count,
    percentile_approx, col, from_json, struct
)
from pyspark.sql.types import (
    StructType, StructField, StringType,
    DoubleType, LongType, TimestampType
)

class ForexSparkStreamProcessor:
    """
    Spark Streaming processor for aggregating forex data
    """
    
    def __init__(self, config_path: str):
        """Initialize Spark Streaming processor with configuration"""
        with open(config_path, 'r') as file:
            self.config = yaml.safe_load(file)
        
        self._setup_logging()
        self._setup_spark()
        self._setup_schema()
    
    def _setup_logging(self):
        """Configure logging"""
        logging.basicConfig(
            level=self.config['logging']['level'],
            format=self.config['logging']['format'],
            filename=self.config['logging']['file']
        )
        self.logger = logging.getLogger('ForexSparkStreamProcessor')
    
    def _setup_spark(self):
        """Initialize Spark session"""
        try:
            self.spark = (SparkSession.builder
                .appName(self.config['spark']['app_name'])
                .master(self.config['spark']['master'])
                .config("spark.streaming.stopGracefullyOnShutdown", "true")
                .getOrCreate())
            
            self.logger.info("Spark session initialized successfully")
        except Exception as e:
            self.logger.error(f"Failed to initialize Spark session: {str(e)}")
            raise
    
    def _setup_schema(self):
        """Define schema for incoming data"""
        self.schema = StructType([
            StructField("symbol", StringType(), True),
            StructField("price", DoubleType(), True),
            StructField("quantity", DoubleType(), True),
            StructField("timestamp", LongType(), True),
            StructField("processing_timestamp", StringType(), True)
        ])
    
    def _create_streaming_query(self, topic: str):
        """Create and return streaming query for given topic"""
        return (self.spark.readStream
            .format("kafka")
            .option("kafka.bootstrap.servers",
                   self.config['kafka']['bootstrap_servers'])
            .option("subscribe", topic)
            .option("startingOffsets", "latest")
            .load()
            .select(from_json(
                col("value").cast("string"),
                self.schema
            ).alias("data"))
            .select("data.*"))
    
    def _process_stream(self, df, topic: str):
        """Process streaming dataframe with windowed aggregations"""
        window_duration = self.config['spark']['window_duration']
        slide_duration = self.config['spark']['slide_duration']
        
        # Calculate aggregations
        aggregations = (df
            .withWatermark("timestamp", window_duration)
            .groupBy(window(col("timestamp"), window_duration, slide_duration))
            .agg(
                avg("price").alias("avg_price"),
                min("price").alias("min_price"),
                max("price").alias("max_price"),
                stddev("price").alias("price_stddev"),
                count("*").alias("trade_count"),
                percentile_approx("price", array(0.25, 0.5, 0.75), 100)
                    .alias("price_percentiles"),
                avg("quantity").alias("avg_quantity"),
                sum("quantity").alias("total_volume")
            ))
        
        # Write results to console (modify for production)
        query = (aggregations.writeStream
            .outputMode("complete")
            .format("console")
            .option("truncate", "false")
            .option("numRows", 100)
            .trigger(processingTime=slide_duration)
            .start())
        
        return query
    
    def start_processing(self, topic: str):
        """Start processing streams for given topic"""
        try:
            self.logger.info(f"Starting stream processing for topic: {topic}")
            
            # Create streaming dataframe
            streaming_df = self._create_streaming_query(topic)
            
            # Process stream and get query
            query = self._process_stream(streaming_df, topic)
            
            # Wait for query termination
            query.awaitTermination()
            
        except Exception as e:
            self.logger.error(f"Error in stream processing: {str(e)}")
            raise
        finally:
            self.shutdown()
    
    def shutdown(self):
        """Clean shutdown of Spark session"""
        try:
            self.spark.stop()
            self.logger.info("Spark session shut down successfully")
        except Exception as e:
            self.logger.error(f"Error during shutdown: {str(e)}")

def main():
    """Main entry point for Spark Streaming processor"""
    processor = ForexSparkStreamProcessor('config.yaml')
    
    # Process BTCUSDT stream as example
    topic = 'forex_btcusdt'
    processor.start_processing(topic)

if __name__ == "__main__":
    main()

In [3]:
import os
import uuid
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, LongType, BooleanType
)
from configs import kafka_config


In [4]:

# Налаштування PySpark для використання Kafka
os.environ[
    'PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'


In [5]:

# Ініціалізація SparkSession
spark = (SparkSession.builder
         .appName("ForexSparkStreamProcessor")
         .master("local[*]")
         .config("spark.sql.debug.maxToStringFields", "200")
         .config("spark.sql.columnNameLengthThreshold", "200")
         .config("spark.streaming.stopGracefullyOnShutdown", "true")
         .config("spark.executor.memory", "4g")
         .config("spark.executor.cores", "4")
         .getOrCreate())


25/01/11 15:51:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:

# Налаштування параметрів вікна
window_duration = "1 minute"
sliding_interval = "30 seconds"


In [7]:

# Схема вхідних даних
schema = StructType([
    StructField("event_type", StringType(), True),
    StructField("event_time", LongType(), True),
    StructField("symbol", StringType(), True),
    StructField("trade_id", LongType(), True),
    StructField("price", DoubleType(), True),
    StructField("quantity", DoubleType(), True),
    StructField("trade_time", LongType(), True),
    StructField("buyer_market_maker", BooleanType(), True),
    StructField("ignore", BooleanType(), True)
])


In [8]:

# Зчитування потоку з Kafka
df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", kafka_config['bootstrap_servers'][0])
      .option("kafka.security.protocol", "SASL_PLAINTEXT")
      .option("kafka.sasl.mechanism", "PLAIN")
      .option("kafka.sasl.jaas.config",
              'org.apache.kafka.common.security.plain.PlainLoginModule required username="admin" password="VawEzo1ikLtrA8Ug8THa";')
      .option("subscribe", 'topic_XRPUSDT')
      .option("startingOffsets", "latest")
      .option("maxOffsetsPerTrigger", "500")  # Обмеження кількості даних на тригер
      .load())


In [9]:

# Обробка даних
processed_df = (df
                .select(from_json(col("value").cast("string"), schema).alias("data"))
                .select(
                    col("data.symbol"),
                    col("data.price"),
                    col("data.quantity"),
                    col("data.trade_time").alias("timestamp")
                )
                .withColumn("event_time", col("timestamp").cast("timestamp"))
                .groupBy(window(col("event_time"), window_duration, sliding_interval))
                .agg(
                    avg("price").alias("avg_price"),
                    min("price").alias("min_price"),
                    max("price").alias("max_price"),
                    stddev("price").alias("price_stddev"),
                    count("*").alias("trade_count"),
                    avg("quantity").alias("avg_quantity"),
                    sum("quantity").alias("total_volume"),
                    percentile_approx("price", array(lit(0.25), lit(0.5), lit(0.75)), 100).alias("price_percentiles")
                ))





In [10]:

# Додавання ключа та підготовка до запису
uuid_udf = udf(lambda: str(uuid.uuid4()), StringType())
output_df = (processed_df
             .withColumn("key", uuid_udf())
             .select(
                 col("key"),
                 to_json(struct(
                     col("window"),
                     col("avg_price"),
                     col("min_price"),
                     col("max_price"),
                     col("price_stddev"),
                     col("trade_count"),
                     col("avg_quantity"),
                     col("total_volume"),
                     col("price_percentiles")
                 )).alias("value")))


In [11]:

# Вивід у консоль для дебагінгу
console_query = (processed_df
                 .writeStream
                 .trigger(processingTime="30 seconds")  # Інтервал тригера
                 .outputMode("update")  # Для потокових даних
                 .format("console")
                 .option("truncate", "false")  # Повний вивід без обрізання
                 .option("numRows", 50)  # Кількість рядків для відображення
                 .start())


25/01/11 15:52:04 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d07f625f-fba5-4ae6-8844-ebbca2c4e374. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/01/11 15:52:04 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:

# Запис у Kafka
kafka_query = (output_df
               .writeStream
               .trigger(processingTime="60 seconds")  # Збільшено інтервал тригера для зменшення навантаження
               .outputMode("update")
               .format("kafka")
               .option("kafka.bootstrap.servers", kafka_config['bootstrap_servers'][0])
               .option("topic", 'output_topic')
               .option("kafka.security.protocol", "SASL_PLAINTEXT")
               .option("kafka.sasl.mechanism", "PLAIN")
               .option("kafka.sasl.jaas.config",
                       'org.apache.kafka.common.security.plain.PlainLoginModule required username="admin" password="VawEzo1ikLtrA8Ug8THa";')
               .option("checkpointLocation", "/tmp/spark_checkpoints")
               .start())


25/01/11 15:52:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/01/11 15:52:07 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


25/01/11 15:52:07 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [15]:

console_query.awaitTermination()
kafka_query.awaitTermination()


25/01/11 16:04:17 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 60000 milliseconds, but spent 137385 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----------------------------------------------+------------------+---------+---------+---------------------+-----------+------------------+------------+------------------------+
|window                                        |avg_price         |min_price|max_price|price_stddev         |trade_count|avg_quantity      |total_volume|price_percentiles       |
+----------------------------------------------+------------------+---------+---------+---------------------+-----------+------------------+------------+------------------------+
|{+57000-10-01 06:05:30, +57000-10-01 06:06:30}|2.3847            |2.3847   |2.3847   |NULL                 |1          |159.0             |159.0       |[2.3847, 2.3847, 2.3847]|
|{+57000-10-01 07:27:00, +57000-10-01 07:28:00}|2.3851            |2.3851   |2.3851   |NULL                 |1          |10.0              |10.0        |[2.3851, 2.3851, 2.3851]|
|{+57000

25/01/11 16:04:45 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 30000 milliseconds, but spent 102458 milliseconds
25/01/11 16:05:53 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 60000 milliseconds, but spent 95692 milliseconds


-------------------------------------------
Batch: 3
-------------------------------------------
+----------------------------------------------+------------------+---------+---------+---------------------+-----------+------------------+------------+------------------------+
|window                                        |avg_price         |min_price|max_price|price_stddev         |trade_count|avg_quantity      |total_volume|price_percentiles       |
+----------------------------------------------+------------------+---------+---------+---------------------+-----------+------------------+------------+------------------------+
|{+57000-10-04 19:45:00, +57000-10-04 19:46:00}|2.3886            |2.3886   |2.3886   |NULL                 |1          |5.0               |5.0         |[2.3886, 2.3886, 2.3886]|
|{+57000-10-01 10:24:00, +57000-10-01 10:25:00}|2.3849            |2.3849   |2.3849   |NULL                 |1          |128.0             |128.0       |[2.3849, 2.3849, 2.3849]|
|{+57000

25/01/11 16:06:13 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 30000 milliseconds, but spent 88899 milliseconds
25/01/11 16:07:11 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 60000 milliseconds, but spent 78698 milliseconds


...

-------------------------------------------
Batch: 6
-------------------------------------------
+----------------------------------------------+---------+---------+---------+------------+-----------+-----------------+------------+------------------------+
|window                                        |avg_price|min_price|max_price|price_stddev|trade_count|avg_quantity     |total_volume|price_percentiles       |
+----------------------------------------------+---------+---------+---------+------------+-----------+-----------------+------------+------------------------+
|{+57000-10-19 17:03:30, +57000-10-19 17:04:30}|2.3916   |2.3916   |2.3916   |0.0         |7          |892.4285714285714|6247.0      |[2.3916, 2.3916, 2.3916]|
|{+57000-10-19 17:03:00, +57000-10-19 17:04:00}|2.3916   |2.3916   |2.3916   |0.0         |7          |892.4285714285714|6247.0      |[2.3916, 2.3916, 2.3916]|
+----------------------------------------------+---------+---------+---------+------------+-----------+

25/01/11 16:10:45 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 30000 milliseconds, but spent 105656 milliseconds


-------------------------------------------
Batch: 7
-------------------------------------------
+----------------------------------------------+------------------+---------+---------+------------+-----------+-----------------+------------+------------------------+
|window                                        |avg_price         |min_price|max_price|price_stddev|trade_count|avg_quantity     |total_volume|price_percentiles       |
+----------------------------------------------+------------------+---------+---------+------------+-----------+-----------------+------------+------------------------+
|{+57000-10-19 17:06:30, +57000-10-19 17:07:30}|2.3914999999999975|2.3915   |2.3915   |0.0         |44         |3.0              |132.0       |[2.3915, 2.3915, 2.3915]|
|{+57000-10-19 17:18:00, +57000-10-19 17:19:00}|2.3914            |2.3914   |2.3914   |NULL        |1          |14.0             |14.0        |[2.3914, 2.3914, 2.3914]|
|{+57000-10-19 17:26:30, +57000-10-19 17:27:30}|2.39140000

25/01/11 16:11:33 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 30000 milliseconds, but spent 47416 milliseconds
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/gamelt/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/gamelt/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 